In [ ]:
# Mac
# brew install poppler
%pip install python-poppler

In [ ]:
%pip install langchain
%pip install openai
%pip install pinecone-client
%pip install unstructured
%pip install unstructured-inference
%pip install "unstructured.PaddleOCR"
%pip install pdfminer
%pip install torch torchvision torchaudio -f https://download.pytorch.org/whl/cu111
%pip install 'git+https://github.com/facebookresearch/detectron2.git@v0.4#egg=detectron2'

In [ ]:
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader
from langchain.vectorstores import Chroma, Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain
import os
import pinecone

In [ ]:
# https://platform.openai.com/
os.environ["OPENAI_API_KEY"] = "your key here"
print(os.environ["OPENAI_API_KEY"])

In [ ]:
loader = UnstructuredPDFLoader("./files/Agatha-Christie.pdf")
# loader = OnlinePDFLoader("https://sherlock-holm.es/stories/pdf/a4/1-sided/stud.pdf")
data = loader.load()

In [ ]:
print(f"You have loaded {len(data)},document(s)")
print(f"There are {len(data[0].page_content)} characters in the document")

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
texts = text_splitter.split_documents(data)

In [ ]:
print(f"There are {len(texts)} chunks of text")

In [ ]:
embeddings = OpenAIEmbeddings(openai_api_key=os.environ["OPENAI_API_KEY"])

In [ ]:
pinecone.init(
    api_key="your key here",
    environment="your environment here",
)
index_name = "langchain1"

In [ ]:
docsearch = Pinecone.from_texts(
  [t.page_content for t in texts], embeddings, index_name=index_name
)

In [ ]:
query = "What is the document about?"
docs = docsearch.similarity_search(query, include_metadata=True)

In [ ]:
docs

In [ ]:
query = "Who is the main character?"
docs = docsearch.similarity_search(query, include_metadata=True)
docs

In [ ]:
llm = OpenAI(temperature=0, openai_api_key=os.environ["OPENAI_API_KEY"])
chain = load_qa_chain(llm, chain_type="stuff")

In [ ]:
query = "What is the document about?"
docs = docsearch.similarity_search(query, include_metadata=True)

In [ ]:
chain.run(input_documents=docs, question=query)

In [ ]:
query = "Who is the main suspect?"
docs = docsearch.similarity_search(query, include_metadata=True)
chain.run(input_documents=docs, question=query)
